In [1]:
# Install necessary libraries
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install tensorflow_datasets
!pip install nltk
!pip install rouge_score


  Obtaining dependency information for protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3 from https://files.pythonhosted.org/packages/a7/ad/bf3f358e90b7e70bf7fb520702cb15307ef268262292d3bdb16ad8ebc815/protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata
  Obtaining dependency information for protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3 from https://files.pythonhosted.org/packages/80/70/dc63d340d27b8ff22022d7dd14b8d6d68b479a003eacdc4507150a286d9a/protobuf-4.23.4-cp310-abi3-win_amd64.whl.metadata
  Using cached protobuf-4.23.4-cp310-abi3-win_amd64.whl.metadata (540 bytes)
Using cached protobuf-4.23.4-cp310-abi3-win_amd64.whl (422 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.68.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.23.4 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for tensorflow_datasets from https://files.pythonhosted.org/packages/43/e6/a85c9a2d8ce72c09c8c8bf231c75e6e12e0e0848df336b28d1ec0beae1fe/tensorflow_datasets-4.9.7-py3-none-any.whl.metadata
  Obtaining dependency information for dm-tree from https://files.pythonhosted.org/packages/e4/c1/522041457444b67125ac9527208bb3148f63d7dce0a86ffa589ec763a10e/dm_tree-0.1.8-cp311-cp311-win_amd64.whl.metadata
  Using cached dm_tree-0.1.8-cp311-cp311-win_amd64.whl.metadata (2.0 kB)
  Obtaining dependency information for immutabledict from https://files.pythonhosted.org/packages/59/56/25ca7b848164b7d93dbd5fc97dd7751700c93e324fe854afbeb562ee2f98/immutabledict-4.2.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for simple-parsing from https://files.pythonhosted.org/packages/f8/62/0194ea1d8245d76bfd2d5c6bebec7f824af00a6e03bec22697d12712fff0/simple_parsing-0.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.29.0 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24970 sha256=77f25d053dc0b80025202e86088866997d88e00a2d1a6991e4ef83965fd5907b
  Stored in directory: c:\users\thaya\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import re
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thaya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [29]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
data = pd.read_csv('TamilSummarization.csv')

# Example: dataset structure [input_text, summary]
input_texts = data['Text'].values
summaries = data['Summary'].values

# Preprocessing the data
def preprocess_text(text):
    # Clean and tokenize the text (remove unwanted characters, punctuation, etc.)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

input_texts = [preprocess_text(text) for text in input_texts]
summaries = [preprocess_text(text) for text in summaries]

# Add special tokens to summaries
def add_special_tokens_to_summaries(summaries):
    return ['<start> ' + summary + ' <end>' for summary in summaries]

summaries_with_special_tokens = add_special_tokens_to_summaries(summaries)

# Define tokenizer for input texts (Tamil paragraphs)
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)

# Define tokenizer for output texts (Summaries with special tokens)
output_tokenizer = Tokenizer(filters='', lower=False)  # Ensure special tokens are not filtered out
output_tokenizer.fit_on_texts(summaries_with_special_tokens)
output_sequences = output_tokenizer.texts_to_sequences(summaries_with_special_tokens)

# Ensure that <start> and <end> are added to the tokenizer's word index
if '<start>' not in output_tokenizer.word_index:
    output_tokenizer.word_index['<start>'] = len(output_tokenizer.word_index) + 1
if '<end>' not in output_tokenizer.word_index:
    output_tokenizer.word_index['<end>'] = len(output_tokenizer.word_index) + 1

# Rebuild the index_word after adding the tokens
output_tokenizer.index_word = {v: k for k, v in output_tokenizer.word_index.items()}

# Check tokenizer word index
print("Output Tokenizer Word Index:")
print(output_tokenizer.word_index)

# Check if "<start>" and "<end>" exist in the word index
assert '<start>' in output_tokenizer.word_index, "Error: '<start>' not found in word index."
assert '<end>' in output_tokenizer.word_index, "Error: '<end>' not found in word index."

# Verify reverse mapping
print("Index to Word Mapping for Output Tokenizer:")
print({v: k for k, v in output_tokenizer.word_index.items()})


# Get maximum sequence lengths
max_input_length = max([len(seq) for seq in input_sequences])
max_output_length = max([len(seq) for seq in output_sequences])

# Padding sequences to ensure uniform length
input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# Print shapes for verification
print(f"Input sequences shape: {input_sequences.shape}")
print(f"Output sequences shape: {output_sequences.shape}")


Output Tokenizer Word Index:
{'<start>': 1, '<end>': 2, 'மற்றும்': 3, 'உதவும்.': 4, 'பல': 5, 'அவசியம்.': 6, 'சக்தி': 7, 'தருகின்றன.': 8, 'நல்ல': 9, 'காற்று': 10, 'உடல்': 11, 'தருகிறது.': 12, 'ஒளி': 13, 'உதவுகிறது.': 14, 'சுற்றுச்சூழல்': 15, 'பறவைகள்': 16, 'நீர்': 17, 'விலங்குகள்': 18, 'உணவு': 19, 'தண்ணீர்': 20, 'பாதுகாக்க': 21, 'உள்ளன.': 22, 'வழங்குகிறது.': 23, 'சூரியன்': 24, 'பூமிக்கு': 25, 'அது': 26, 'உடலுக்கு': 27, 'இரவில்': 28, 'மக்கள்': 29, 'மழை': 30, 'பெரிய': 31, 'சூரிய': 32, 'சுத்தமான': 33, 'ஆரோக்கியத்துக்கு': 34, 'விவசாயத்திற்கு': 35, 'சுத்தம்': 36, 'நோய்களைத்': 37, 'மகிழ்ச்சி': 38, 'மூலம்': 39, 'மாணவர்களுக்கு': 40, 'ஐ': 41, 'வழங்கும்.': 42, 'பூமி': 43, 'கொண்டுள்ளன.': 44, 'சந்திரன்': 45, 'காப்பாற்ற': 46, 'அறிவை': 47, 'ஆரோக்கியத்தை': 48, 'மேம்படுத்தும்.': 49, 'வாழ': 50, 'மேலும்': 51, 'மரங்கள்': 52, 'பாதுகாப்புக்கு': 53, 'விவசாயிகள்': 54, 'சூழல்': 55, 'காய்கறிகள்': 56, 'நண்பன்.': 57, 'உதவுகின்றன.': 58, 'அறிவு': 59, 'பாதுகாப்பாக': 60, 'சத்தான': 61, 'நல்லது.': 62, 'பசுமை': 63, 'பரி

In [30]:
# Split the dataset into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(input_sequences, output_sequences, test_size=0.2, random_state=42)

# Convert to numpy arrays
X_train, X_val, y_train, y_val = np.array(X_train), np.array(X_val), np.array(y_train), np.array(y_val)


In [31]:
# Define the model
def create_model(vocab_size_input, vocab_size_output, max_input_len, max_output_len, embedding_dim=128):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(256, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(max_output_len,))
    decoder_embedding = Embedding(input_dim=vocab_size_output, output_dim=embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    
    # Attention mechanism
    attention = Attention(use_scale=True)([decoder_outputs, encoder_outputs])
    decoder_combined_context = Concatenate()([decoder_outputs, attention])

    # Dense layer to predict the output word
    decoder_dense = Dense(vocab_size_output, activation='softmax')
    decoder_final_output = decoder_dense(decoder_combined_context)

    # Create the model
    model = Model([encoder_inputs, decoder_inputs], decoder_final_output)

    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [32]:
# Get vocab sizes for input and output tokens
vocab_size_input = len(input_tokenizer.word_index) + 1
vocab_size_output = len(output_tokenizer.word_index) + 1

# Create the model
model = create_model(vocab_size_input, vocab_size_output, max_input_length, max_output_length)

# Prepare the decoder input (shifted summary texts)
decoder_input_data = np.zeros_like(y_train)
decoder_input_data[:, 1:] = y_train[:, :-1]

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    [X_train, decoder_input_data], np.expand_dims(y_train, -1),
    validation_data=([X_val, np.zeros_like(y_val)], np.expand_dims(y_val, -1)),
    epochs=30, batch_size=64, callbacks=[early_stopping]
)


Epoch 1/30
2/2 [==============================] - 9s 2s/step - loss: 6.3060 - accuracy: 0.3034 - val_loss: 6.1179 - val_accuracy: 0.6727
Epoch 2/30
2/2 [==============================] - 1s 750ms/step - loss: 6.0660 - accuracy: 0.6674 - val_loss: 5.5923 - val_accuracy: 0.6727
Epoch 3/30
2/2 [==============================] - 1s 728ms/step - loss: 5.3755 - accuracy: 0.6674 - val_loss: 3.1031 - val_accuracy: 0.6727
Epoch 4/30
2/2 [==============================] - 1s 748ms/step - loss: 2.8380 - accuracy: 0.6674 - val_loss: 2.3772 - val_accuracy: 0.6727
Epoch 5/30
2/2 [==============================] - 1s 732ms/step - loss: 2.3314 - accuracy: 0.6674 - val_loss: 2.2213 - val_accuracy: 0.6727
Epoch 6/30
2/2 [==============================] - 1s 593ms/step - loss: 2.1100 - accuracy: 0.6674 - val_loss: 2.1198 - val_accuracy: 0.6727
Epoch 7/30
2/2 [==============================] - 1s 500ms/step - loss: 2.0559 - accuracy: 0.6674 - val_loss: 2.1107 - val_accuracy: 0.7167
Epoch 8/30
2/2 [=======

In [33]:
# Evaluate the model
loss, accuracy = model.evaluate([X_val, np.zeros_like(y_val)], np.expand_dims(y_val, -1))
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


1/1 [==============================] - 0s 139ms/step - loss: 2.1027 - accuracy: 0.7227
Validation Loss: 2.1027143001556396
Validation Accuracy: 0.7227272987365723


In [36]:
# Save the model
model.save('tamil_summarization_model.h5')

C:\Users\thaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
# Save the model in the SavedModel format
model.save('tamil_summarization_model')

# Load the model later
loaded_model = tf.keras.models.load_model('tamil_summarization_model')


INFO:tensorflow:Assets written to: tamil_summarization_model\assets


INFO:tensorflow:Assets written to: tamil_summarization_model\assets
